## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table_GM = 'GM_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table_GM,
                'filter' : "AND item_sub_department_name = '400 GROWING MEDIA' "})

snowflake_tools - 2025-07-08 17:02:03.956891+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-07-08 17:02:17.116256+10:00 - Saved token with '2025-07-08 21:02:17+10:00' expiry...


## Demographic Segments (AU)

### Overall index

In [6]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'"
    }
)

In [7]:
print(f"Total sales: ${ds['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(ds['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(ds['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $160,691,891.00
Total quantity: 18,538,506
Total customers: 2,592,008


In [8]:
ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,DEMOGRAPHIC_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Homeowner older kids,12658140.01,0.078773,0.83
1,Homeowner younger kids,21650028.46,0.134730,0.72
2,Older homeowner,87747275.73,0.546059,1.31
3,Renter no kids,9191033.47,0.057197,0.65
4,Renter with kids,5073012.15,0.031570,0.52
5,Unclassifiable,1434624.35,0.008928,1.31
6,Younger homeowner,22937776.83,0.142744,0.99


$161M in sales, 54% comes from Older homeowner
- Older homeowner over indexed - loves gardening, have large garden
- Homwowner younger kids and renters under indexed - no garden?

### Class index

In [9]:
GM_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [10]:
summary = (
    GM_ds[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR MULCH BLOCK HYDRO,1155603.63,60511.0,30355,0.007191,0.003264,0.006839
7,500 COIR POTTING MIX BLOCK,708514.12,84519.0,49384,0.004409,0.004559,0.011126
14,500 COMPOSTS SOIL CONDITIONERS,29642027.43,4166671.0,881094,0.184465,0.224758,0.198498
21,500 MANURE,5803884.42,1101361.0,293716,0.036118,0.059409,0.066170
28,500 MULCHES,39012103.21,3291411.0,742063,0.242776,0.177545,0.167177
35,500 POTTING MIXES,76869003.63,8938231.0,1948306,0.478363,0.482144,0.438926
42,500 SMALL BAG POTTING MIX,7500754.56,895802.0,493880,0.046678,0.048321,0.111264


In [11]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $160,691,891.00
Total quantity: 18,538,506
Total customers: 4,438,798


In [12]:
GM_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR MULCH BLOCK HYDRO,Homeowner older kids,100865.04,0.087283,1.108041
1,500 COIR MULCH BLOCK HYDRO,Homeowner younger kids,180888.60,0.156532,1.161817
2,500 COIR MULCH BLOCK HYDRO,Older homeowner,543161.02,0.470024,0.860756
3,500 COIR MULCH BLOCK HYDRO,Renter no kids,84448.50,0.073077,1.277652
4,500 COIR MULCH BLOCK HYDRO,Renter with kids,42079.39,0.036413,1.153423
5,500 COIR MULCH BLOCK HYDRO,Unclassifiable,8000.41,0.006923,0.775459
6,500 COIR MULCH BLOCK HYDRO,Younger homeowner,196160.67,0.169747,1.189175
7,500 COIR POTTING MIX BLOCK,Homeowner older kids,55588.85,0.078458,0.996009
8,500 COIR POTTING MIX BLOCK,Homeowner younger kids,97895.37,0.138170,1.025532
9,500 COIR POTTING MIX BLOCK,Older homeowner,343730.03,0.485142,0.888442


48% sales comes from 500 POTTING MIXES, 24% from 500 MULCHES, 18% from 500 COMPOSTS SOIL CONDITIONERS, the rest 10% are minimal...	
- Renter no kids under indexed in 500 MANURE, 500 MULCHES
- Renters over indexed in 500 COIR MULCH BLOCK HYDRO, 500 COIR POTTING MIX BLOCK, 500 SMALL BAG POTTING MIX
- Homeowner younger kids over indexed in 500 COMPOSTS SOIL CONDITIONERS

## Non-Commercial (NZ)

### Class index

In [13]:
non_commercial = con.read_sql_file_into_pandas(
    file="sql/non-commercial-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ'
    }
)

NameError: name 'table' is not defined

In [ ]:
non_commercial

In [ ]:
print(f"Total sales: ${non_commercial['TOTAL_SALES'].sum():,.2f}")
print(f"Total quantity: {int(non_commercial['TOTAL_QUANTITY'].sum()):,}")

## Commercial Industry Segment (NZ)

### Overall index

In [16]:
cis = con.read_sql_file_into_pandas(
    file="sql/commercial-industry-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'",
        'country' : 'NZ'
    }
)

In [17]:
print(f"Total sales: ${cis['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(cis['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(cis['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $4,075,711.62
Total quantity: 474,806
Total customers: 34,878


In [25]:
n = 8
top_n = cis.nlargest(n, "RANGE_SALES").copy()
top_n[["INDUSTRY_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,INDUSTRY_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Professional Computer and Scientific Services,672372.09,0.164970,2.00
1,Personal and Other Services,574593.10,0.140980,2.33
2,Rental Hiring and Real Estate Services,329816.04,0.080922,0.93
3,Retail and Wholesale Trade,309192.48,0.075862,1.49
4,Residential Builder,308458.70,0.075682,0.30
5,Manufacturing,294825.51,0.072337,1.40
6,Site Preparation and Development,214062.55,0.052522,0.74
7,Landscaper and Gardening Services,204666.26,0.050216,1.49


In [26]:
print(f"Top {n} segments - sales share: {top_n['RANGE_SALES_SHARE'].sum():.2%}")
print(f"Top {n} segments - quantity share: {top_n['RANGE_QUANTITY_SHARE'].sum():.2%}")
print(f"Top {n} segments - customer share: {top_n['RANGE_CUSTOMER_SHARE'].sum():.2%}")
print(f"Top {n} segments - size share: {top_n['SEGMENT_SIZE_SHARE'].sum():.2%}")

Top 8 segments - sales share: 71.35%
Top 8 segments - quantity share: 71.31%
Top 8 segments - customer share: 64.60%
Top 8 segments - size share: 45.98%


### Class index

In [20]:
# Get data
GM_is = con.read_sql_file_into_pandas(
    file="sql/2-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': n, 
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [21]:
summary = (
    GM_is[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR POTTING MIX BLOCK,15123.08,2526.0,473,0.004965,0.007123,0.011741
10,500 COMPOSTS SOIL CONDITIONERS,917027.36,130025.0,12620,0.301037,0.366639,0.313268
20,500 MULCHES,495346.37,52570.0,5133,0.162610,0.148235,0.127417
30,500 POTTING MIXES,1481996.10,154561.0,17261,0.486501,0.435825,0.428472
40,500 SMALL BAG POTTING MIX,136739.31,14958.0,4798,0.044888,0.042178,0.119101


In [22]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $3,046,232.22
Total quantity: 354,640
Total customers: 40,285


In [23]:
GM_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR POTTING MIX BLOCK,Commercial Builder,117.44,0.007766,0.394038
1,500 COIR POTTING MIX BLOCK,Landscaper and Gardening Services,1362.89,0.090120,1.341335
2,500 COIR POTTING MIX BLOCK,Manufacturing,1852.68,0.122507,1.265780
3,500 COIR POTTING MIX BLOCK,Owner Builder,624.55,0.041298,1.608503
4,500 COIR POTTING MIX BLOCK,Personal and Other Services,2756.83,0.182293,0.966434
5,500 COIR POTTING MIX BLOCK,Professional Computer and Scientific Services,2996.74,0.198157,0.897764
6,500 COIR POTTING MIX BLOCK,Rental Hiring and Real Estate Services,1265.87,0.083705,0.773108
7,500 COIR POTTING MIX BLOCK,Residential Builder,1999.29,0.132201,1.305574
8,500 COIR POTTING MIX BLOCK,Retail and Wholesale Trade,1655.48,0.109467,1.078494
9,500 COIR POTTING MIX BLOCK,Site Preparation and Development,491.31,0.032487,0.462315


## Drop item range table

In [24]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table_GM}
    )